In [1]:
!nvidia-smi

Sun Dec 24 02:44:30 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 537.58       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4060 Ti     On  | 00000000:2B:00.0  On |                  N/A |
|  0%   46C    P5              20W / 160W |   1531MiB /  8188MiB |     40%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import torch
from torch import nn
torch.device('cpu'), torch.device('cuda'), torch.device('cuda:1')

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=1))

In [4]:
torch.cuda.device_count()

1

In [5]:
def try_gpu(i=0):
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    devices = [torch.device(f'cuda:{i}')
                for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

In [6]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

In [7]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [19]:
# 当矩阵较小，cpu的速度会比gpu要快，但是当矩阵维度很大时，使用gpu更快
import torch
import time

# Set the device to CPU and GPU
device_cpu = torch.device("cpu")
device_gpu = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Generate two 100x100 matrices of random numbers for both CPU and GPU
matrix1_cpu = torch.rand(1000, 1000, device=device_cpu)
matrix2_cpu = torch.rand(1000, 1000, device=device_cpu)

matrix1_gpu = torch.rand(1000, 1000, device=device_gpu)
matrix2_gpu = torch.rand(1000, 1000, device=device_gpu)

# Measure the time taken to perform 1000 matrix multiplications on CPU
start_time_cpu = time.time()
for _ in range(1000):
    result_cpu = torch.matmul(matrix1_cpu, matrix2_cpu)
end_time_cpu = time.time()
total_time_cpu = end_time_cpu - start_time_cpu

# Measure the time taken to perform 1000 matrix multiplications on GPU
torch.cuda.synchronize()  # Ensure CUDA operations are synchronized
start_time_gpu = time.time()
for _ in range(1000):
    result_gpu = torch.matmul(matrix1_gpu, matrix2_gpu)
torch.cuda.synchronize()  # Ensure CUDA operations are synchronized before measuring the end time
end_time_gpu = time.time()
total_time_gpu = end_time_gpu - start_time_gpu

total_time_cpu, total_time_gpu

(4.985057592391968, 0.30353307723999023)